In [ ]:
import pandas as pd
import os

In [ ]:
file = pd.read_csv('messages_export.csv')

In [ ]:
file.head()

In [ ]:
output = "chats"


dataset needs to reversed, rn it is latest - first
we need increasing order to build context as we go down the rows

In [ ]:
id = 1
for chat_id, group_df in file.groupby("chat_id"):
    filename = f"chat_{id}.csv"
    filepath = os.path.join(output, filename)
    group_df = group_df.iloc[::-1]
    group_df.to_csv(filepath, index=False)
    id += 1
    print(f"Saved: {filepath}")

In [ ]:
chat_1 = pd.read_csv('chats/chat_1.csv')

In [ ]:
chat_1.head()

In [ ]:
chat_1['from_number'].unique()

In [ ]:
chat_2 = pd.read_csv('chats/chat_2.csv')


In [ ]:
chat_2['from_number'].unique()

In [ ]:
chat_1.info()

In [ ]:
chat_1['text_body'][:10]

In [ ]:
for i in range(20):
    print(chat_1['from_number'][i],"--",chat_1['text_body'][i],"\n")

In [ ]:
chat_1_new = chat_1[['message_id','type','text_body','from_number','media_url','context','created_at']]

In [ ]:
chat_1_new

In [ ]:
chat_1_new['created_at'] = pd.to_datetime(chat_1_new['created_at'])

In [ ]:
import re

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", temperature=0)


In [ ]:
# check if user is talking about themselves
# determine if it is anything important
# future scope - integrate context
# future scope - check with before messages
def user_facts(text):
    if bool(re.search(r"\b(I|my|me|we)\b", text, re.IGNORECASE)):
        extracted_fact = llm.invoke(f'your task is to determine if the following message gives any information about the user {text}. If it does, write it in simple manner. If it does not, simply write no').content
        if re.fullmatch(r"\b(no)\b", extracted_fact, re.IGNORECASE):
            print("Not useful")
            return None
        else:
            print(extracted_fact)
            return extracted_fact
            

In [ ]:
columns_user = ['message_id','from_number','text_body','facts','created_at']

In [ ]:
user_facts_csv = pd.DataFrame(columns=columns_user)

In [ ]:
user_facts(chat_1_new['text_body'][0])

In [ ]:
chat_1_new['text_body'] = chat_1_new['text_body'].astype(str)

i need id of every message with info to cross check so i need to iterate every message


In [ ]:
for i in range(len(chat_1_new)):
    response = user_facts(chat_1_new['text_body'][i])
    if response != 'Not useful' or response != 'No':
        user_fact_row = {
            'message_id' : chat_1_new['message_id'][i],
            'from_number' : chat_1_new['from_number'][i],
            'text_body' : chat_1_new['text_body'][i],
            'facts' : response,
            'created_at' : chat_1_new['created_at'][i]
								}
        user_facts_csv = pd.concat([user_facts_csv,pd.DataFrame([user_fact_row])],ignore_index = True)

In [ ]:
user_facts_csv['facts'].unique()

In [ ]:
user_facts_csv = user_facts_csv[user_facts_csv['facts'].notnull()]

In [ ]:
user_facts_csv.to_csv("users_new.csv", index = False)

In [ ]:
user_facts_csv.info()

In [ ]:
import pandas as pd
import re
import uuid
from datetime import datetime


In [ ]:

def extract_task_request(text):
    """
    Determine if the message contains a task request
    """
    # Look for task-indicating patterns
    task_patterns = [
        r"\b(can you|could you|please|help me|I need|do this|make this|create|build|fix|solve)\b",
        r"\b(task|todo|assignment|project|work|job)\b",
        r"\?",  # Questions often indicate requests
        r"\b(urgent|asap|deadline|by tomorrow|need it)\b"
    ]
    
    has_task_indicators = any(re.search(pattern, text, re.IGNORECASE) for pattern in task_patterns)
    
    if has_task_indicators:
        # Use LLM to determine if it's actually a task request
        extracted_task = llm.invoke(
            f'''Analyze this message to determine if the user is requesting a task to be performed: "{text}"
            
            If this is a task request, respond with:
            1. A brief title for the task (max 50 chars)
            2. A description of what needs to be done
            3. Any deadline or urgency mentioned
            
            Format your response as:
            TITLE: [task title]
            DESCRIPTION: [what needs to be done]
            DEADLINE: [any deadline mentioned or "None"]
            
            If this is NOT a task request, simply respond with "NO_TASK"'''
        ).content
        
        if "NO_TASK" in extracted_task.upper():
            return None
        else:
            return extracted_task
    return None


In [ ]:

def extract_task_update(text, existing_tasks):
    """
    Check if the message contains updates about existing tasks
    """
    if not existing_tasks:
        return None
    
    # Create a summary of existing tasks for context
    task_context = "\n".join([f"Task {i+1}: {task['title']}" for i, task in enumerate(existing_tasks)])
    
    update_response = llm.invoke(
        f'''Given these existing tasks:
        {task_context}
        
        Analyze this message: "{text}"
        
        Does this message provide an update, progress report, completion status, or modification request for any of the existing tasks?
        
        If YES, respond with:
        TASK_NUMBER: [which task number this relates to]
        UPDATE_TYPE: [progress/completion/modification/question]
        UPDATE: [brief description of the update]
        
        If NO, respond with "NO_UPDATE"'''
    ).content
    
    if "NO_UPDATE" in update_response.upper():
        return None
    else:
        return update_response


In [ ]:

# Initialize dataframes
task_requests_df = pd.DataFrame(columns=[
    'task_id', 'message_id', 'from_number', 'text_body', 'task_title', 
    'task_description', 'deadline', 'status', 'created_at'
])

task_updates_df = pd.DataFrame(columns=[
    'update_id', 'task_id', 'message_id', 'from_number', 'text_body', 
    'update_type', 'update_description', 'created_at'
])

# Track tasks as we process messages
active_tasks = []

# Process messages chronologically
# chat_1_sorted = chat_1_new.sort_values('created_at').reset_index(drop=True)

for i in range(len(chat_1_new)):
    message_text = chat_1_new['text_body'][i]
    from_number = chat_1_new['from_number'][i]
    message_id = chat_1_new['message_id'][i]
    created_at = chat_1_new['created_at'][i]
    
    # Skip empty messages
    if pd.isna(message_text) or message_text.strip() == '':
        continue
    
    print(f"Processing message {i+1}/{len(chat_1_new)}: {message_text[:50]}...")
    
    # Check for new task requests
    task_response = extract_task_request(message_text)
    
    if task_response:
        # Generate unique task ID
        task_id = str(uuid.uuid4())[:8]
        
        # Parse the LLM response
        lines = task_response.split('\n')
        task_title = "Untitled Task"
        task_description = message_text[:100] + "..."
        deadline = "None"
        
        for line in lines:
            if line.startswith('TITLE:'):
                task_title = line.replace('TITLE:', '').strip()
            elif line.startswith('DESCRIPTION:'):
                task_description = line.replace('DESCRIPTION:', '').strip()
            elif line.startswith('DEADLINE:'):
                deadline = line.replace('DEADLINE:', '').strip()
        
        # Add to task requests dataframe
        task_row = {
            'task_id': task_id,
            'message_id': message_id,
            'from_number': from_number,
            'text_body': message_text,
            'task_title': task_title,
            'task_description': task_description,
            'deadline': deadline,
            'status': 'requested',
            'created_at': created_at
        }
        
        task_requests_df = pd.concat([task_requests_df, pd.DataFrame([task_row])], ignore_index=True)
        
        # Add to active tasks for tracking
        active_tasks.append({
            'task_id': task_id,
            'title': task_title,
            'requester': from_number
        })
        
        print(f"✅ New task identified: {task_title}")
    
    # Check for updates on existing tasks
    if active_tasks:
        update_response = extract_task_update(message_text, active_tasks)
        
        if update_response:
            # Parse the update response
            lines = update_response.split('\n')
            task_number = None
            update_type = "progress"
            update_description = message_text[:100] + "..."
            
            for line in lines:
                if line.startswith('TASK_NUMBER:'):
                    try:
                        task_number = int(line.replace('TASK_NUMBER:', '').strip()) - 1
                    except:
                        pass
                elif line.startswith('UPDATE_TYPE:'):
                    update_type = line.replace('UPDATE_TYPE:', '').strip()
                elif line.startswith('UPDATE:'):
                    update_description = line.replace('UPDATE:', '').strip()
            
            if task_number is not None and 0 <= task_number < len(active_tasks):
                related_task_id = active_tasks[task_number]['task_id']
                
                # Generate unique update ID
                update_id = str(uuid.uuid4())[:8]
                
                # Add to task updates dataframe
                update_row = {
                    'update_id': update_id,
                    'task_id': related_task_id,
                    'message_id': message_id,
                    'from_number': from_number,
                    'text_body': message_text,
                    'update_type': update_type,
                    'update_description': update_description,
                    'created_at': created_at
                }
                
                task_updates_df = pd.concat([task_updates_df, pd.DataFrame([update_row])], ignore_index=True)
                
                print(f"📝 Task update found for: {active_tasks[task_number]['title']}")
                
                # Update task status if completed
                if 'completion' in update_type.lower() or 'complete' in update_description.lower():
                    task_requests_df.loc[task_requests_df['task_id'] == related_task_id, 'status'] = 'completed'

# Display results
print(f"\n🎯 Found {len(task_requests_df)} task requests")
print(f"📝 Found {len(task_updates_df)} task updates")

# Show summary
if not task_requests_df.empty:
    print("\n=== TASK REQUESTS SUMMARY ===")
    for _, task in task_requests_df.iterrows():
        print(f"Task ID: {task['task_id']}")
        print(f"Title: {task['task_title']}")  
        print(f"Requester: {task['from_number']}")
        print(f"Status: {task['status']}")
        print(f"Created: {task['created_at']}")
        
        # Show related updates
        related_updates = task_updates_df[task_updates_df['task_id'] == task['task_id']]
        if not related_updates.empty:
            print(f"Updates: {len(related_updates)}")
            for _, update in related_updates.iterrows():
                print(f"  - {update['update_type']}: {update['update_description'][:50]}...")
        print("-" * 50)

# Save to CSV files
task_requests_df.to_csv('task_requests.csv', index=False)
task_updates_df.to_csv('task_updates.csv', index=False)

print(f"\n💾 Results saved to 'task_requests.csv' and 'task_updates.csv'")

In [ ]:
task_updates_df.info()

In [ ]:
import pandas as pd 

facts = pd.read_csv('user_facts_export.csv')


In [ ]:
facts.info()

In [ ]:

grouped_facts = facts.groupby('from_number')['fact'].apply(list).reset_index()


In [ ]:
def summarize_user_facts(facts_list):
    prompt = f"Here are some facts about a user: {facts_list}. Summarize all the key information in a concise paragraph."
    summary = llm.invoke(prompt).content
    return summary

grouped_facts['user_summary'] = grouped_facts['fact'].apply(summarize_user_facts)


In [ ]:
grouped_facts.info()

In [ ]:
grouped_facts['user_summary'][0]